# Neural Engineering Framework

In this notebook the NEF networks needed for Figure 2 are run. Developed for Python3.

In [1]:
import pandas as pd
import numpy as np
import nengo

In [2]:
def phaseIn(t, x):
    t *= 2
    frac = np.clip(t%1.0 - 0.15, 0, 0.3) / 0.3
    return frac*x

signal = np.array([[0.2, 0.6, 0.0, -0.7, -0.2], [0.0, -0.3, 0.2, 0.0, 0.5]]).T
def drive(t):
    t *= 2
    frac = np.clip(t%1.0 - 0.15, 0, 0.3) / 0.3
    return (1-frac)*signal[np.clip(int(t%5), 0, 4)]

def runNEFNetwork(encoders=None):
    model = nengo.Network(label="Integrator")
    noiseProcess = nengo.processes.WhiteNoise(dist=nengo.dists.Gaussian(0, 7e-3), seed=1)
    
    N = 1000
    D = 2
    with model:
        if encoders is None:
            A = nengo.Ensemble(N, dimensions=D, max_rates=nengo.dists.Uniform(80,120))
        else:
            A = nengo.Ensemble(N, dimensions=D, encoders=encoders)
        tau = 0.01 #Synaptic time constant is 10 ms
        input = nengo.Node(drive, 0, 2)
        input_connection = nengo.Connection(input, A, synapse=tau)
        node = nengo.Node(phaseIn, 2, 2)
        recurrent_connection = nengo.Connection(A, node, synapse=tau)
        feedback_connection = nengo.Connection(node, A)
        A_probe = nengo.Probe(A, synapse=tau)
        spikes_probe = nengo.Probe(A.neurons)
    sim = nengo.Simulator(model, progress_bar=True)
    sim.run(2.5, True)
    K = sim.data[A].encoders
    phi = sim.data[recurrent_connection].weights
    spikes = sim.data[spikes_probe]
    x = sim.data[A_probe]
    return K, phi, spikes, x

Run the unperturbed network

In [3]:
K, phi, spikes, x = runNEFNetwork()
spikes = np.vstack(np.nonzero(spikes)).T

Build finished in 0:00:02.                                                                                                                                              
Simulation finished in 0:00:03.                                                                                                                                         


Write the unperturbed network data to a pytable-file for later analysis

In [4]:
filename = "../generatedData/fig2.h5"

In [5]:
pd.DataFrame(K, columns=["K1", "K2"]).to_hdf(filename, "/nef/original/K")
pd.DataFrame(phi.T, columns=["phi1", "phi2"]).to_hdf(filename, "/nef/original/phi")
pd.DataFrame(spikes, columns=["time", "neuron"]).eval("time = time / 1000").to_hdf(filename, "/nef/original/spikes")
pd.DataFrame(x, columns=["x1", "x2"]).to_hdf(filename, "/nef/original/x")

Run and save the perturbed network

In [6]:
K, phi, spikes, x = runNEFNetwork(K[:,(1,0)])
spikes = np.vstack(np.nonzero(spikes)).T

Build finished in 0:00:02.                                                                                                                                              
Simulation finished in 0:00:04.                                                                                                                                         


In [7]:
pd.DataFrame(K, columns=["K1", "K2"]).to_hdf(filename, "/nef/perturbed/K")
pd.DataFrame(phi.T, columns=["phi1", "phi2"]).to_hdf(filename, "/nef/perturbed/phi")
pd.DataFrame(spikes, columns=["time", "neuron"]).eval("time = time / 1000").to_hdf(filename, "/nef/perturbed/spikes")
pd.DataFrame(x, columns=["x1", "x2"]).to_hdf(filename, "/nef/perturbed/x")